In [1]:
import io, os, uuid, yaml, urllib
import numpy as np
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from src.pipeline import utils, azure_tools
import sqlalchemy


In [2]:
def format_file(df):
    df_clean = df.copy()

    #   Change column name
    df_clean.columns.values[2] = 'company_code_id'
    #   Reset index
    df_clean = df_clean.reset_index(drop = True)
    #   Fill all NANs with 0
    df_clean = df_clean.fillna(0)
    #   Remove the "Result" line
    df_clean = df_clean[df_clean['Company Code'] != 'Result']
    #   Split fiscal year and order data frame
    dates = pd.DataFrame(data = df_clean['Fiscal year/period'].astype(str).str.split(pat = '.', expand = True).astype(int).values, columns = ['month', 'year'])
    df_clean.insert(
        loc = 0,
        column = 'date',
        value = dates.year.astype(str).str.ljust(4,'0')+dates.month.astype(str).str.rjust(2,'0')+"01")

    df_clean = df_clean.astype({
        'Company Code':str,
        'Commercial Name': str,
        'BPC: Customer': str,
        'BPC: Product': str,
        'Ship to party': str,
        'Material': str
    })

    return df_clean

Se cargan las credenciales

In [3]:
credentials = utils.load_credentials("blob_storage")

Código de prueba para la Azure Function

In [4]:
# blob  = BlobClient.from_connection_string(credentials['conn_string'], container_name = "reports-ingredion", blob_name="test Brazil.xlsm")
# with open("./test Brazil P&L.xlsm", "rb") as file:
#     blob.upload_blob(file)

In [5]:
# container_client = azure_tools.get_container_client("reports-ingredion",credentials['conn_string'])
# blob_list = azure_tools.get_blob_list(container_client)
# container_client.delete_blob("test Brazil.xlsm")

Se obtienen la lista de archivos en *Raw*

In [6]:
container_input = "raw-data"
container_client = azure_tools.get_container_client(container_input,credentials['conn_string'])

In [7]:
# name of the existing container
blob_list = azure_tools.get_blob_list(container_client)

In [8]:
len(blob_list)

541

In [9]:
blob_list_us = [i for i in blob_list if "US/Actuals" in i]

In [10]:
len(blob_list_us)

72

In [11]:
blob_list_us[21:40]

['US/Actuals/04-2019 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/04-2020 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/04-2021 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI) (1).xlsm',
 'US/Actuals/05-2016 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/05-2017 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/05-2018 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/05-2019 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/05-2020 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/05-2021 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI) (1).xlsm',
 'US/Actuals/06-2016 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/06-2017 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/06-2018 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm',
 'US/Actuals/06-2019 US P&L by Customer & Product(ZBW_ZA

In [12]:
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=vm-poc-dev-eastus-001.database.windows.net;"
                                 "DATABASE=ingredion;"
                                 "UID=admindbuser;"
                                 "PWD=Innovative.Solutions2021")
#engine = sqlalchemy.create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
for f in blob_list_us[21:40]:
    country, type, file_name = f.split("/")
    stream = container_client.download_blob(f)
    file = io.BytesIO()
    stream.readinto(file)
    df = pd.read_excel(file,
                       sheet_name="Data",
                       skiprows=range(0, 15),
                       usecols = "G:BC")
    print("Limpiando: ", file_name)
    df = format_file(df)
    df.insert(loc=0, column="Country", value=country)
    df.insert(loc=0, column="Type", value=type)
    df.insert(loc=0, column="File", value=file_name)

    table_name = "pl_data_" + type.lower()
    print("Subiendo: ", file_name)
    engine = sqlalchemy.create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
    df.to_sql(table_name, con=engine, schema='clean', if_exists='append', index=False)
    print("Listo: ", file_name)
    print("------------------------------")


Limpiando:  04-2019 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm
Subiendo:  04-2019 US P&L by Customer & Product(ZBW_ZANDTM001_Q00NI).xlsm


OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (0) (SQLEndTran)')
(Background on this error at: http://sqlalche.me/e/13/e3q8)

## Read data

In [ ]:
container_client = azure_tools.get_container_client(container_output,credentials['conn_string'])
blob_list = azure_tools.get_blob_list(container_client)
#container_client.delete_blob(file)

In [ ]:
len(blob_list)

In [ ]:
# for file in blob_list:
#     container_client.delete_blob(file)

In [ ]:
df_test = azure_tools.read_transform_pl(container_output,credentials['conn_string'])

In [ ]:
df_test.shape

In [ ]:
df_test.info()

In [ ]:
container_client = azure_tools.get_container_client(container_output, credentials["conn_string"])

In [ ]:
import pyarrow.parquet as pq

In [ ]:
ds = pq.ParquetDataset("https://storingrediondeveastus.blob.core.windows.net/intermediate-ingredion/")